In [ ]:
'''
In this file, I did:
1. Used the pickle extracted from Stock_Plotting_v1_(Demo).ipydb for stock data and backtest Stratey


'''

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import backtrader as bt
import datetime  # For datetime objects


In [9]:
# Create a Stratey // Buy Executed in another day morning open price
class TestStrategy(bt.Strategy):

    params = (
        ('maperiod', 25), # moving average period 
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        if self.order:
            return
        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:
                # BUY
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.dataclose[0] < self.sma[0]:
                # SELL
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()


if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    # Create a Data (the pickle from Stock_Plotting_v1_(Demo).ipydb)
    df = pd.read_pickle("sp500_stocks/AAPL_av.pickle")
    df.index = pd.to_datetime(df.index)
    df.rename(columns={ '1. open':'open',
                    '2. high':'high',
                    '3. low':'low',
                    '4. close':'close',
                    '5. adjusted close':'adjusted close',
                    '6. volume':'volume',
                    '7. dividend amount':'dividend amount',
                    '8. split coefficient':'split coefficient'}, 
                 inplace=True)
    df = pd.read_pickle("sp500_stocks/AMZN_av.pickle")
    df.index = pd.to_datetime(df.index)
    #df column need to rename to fit the backtrader data detection of columns
    df.rename(columns={ '1. open':'open',
                    '2. high':'high',
                    '3. low':'low',
                    '4. close':'close',
                    '5. adjusted close':'adjusted close',
                    '6. volume':'volume',
                    '7. dividend amount':'dividend amount',
                    '8. split coefficient':'split coefficient'}, 
                 inplace=True)
    data = bt.feeds.PandasData(dataname=df, openinterest=None)
    
    # Add strategy using class TestStrategy(bt.Strategy)
    cerebro.addstrategy(TestStrategy)
    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    
    # Set the initial cash for portfolio
    cerebro.broker.setcash(100000.0)
    
    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)
    
    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    
    
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00
1998-02-06, Close, 58.88
1998-02-06, BUY CREATE, 58.88
1998-02-09, Close, 58.25
1998-02-10, Close, 63.63
1998-02-11, Close, 63.50
1998-02-12, Close, 63.13
1998-02-13, Close, 62.25
1998-02-17, Close, 61.75
1998-02-18, Close, 60.88
1998-02-19, Close, 62.75
1998-02-20, Close, 63.25
1998-02-23, Close, 64.88
1998-02-24, Close, 64.88
1998-02-25, Close, 65.88
1998-02-26, Close, 72.75
1998-02-27, Close, 77.00
1998-03-02, Close, 76.25
1998-03-03, Close, 71.75
1998-03-04, Close, 73.44
1998-03-05, Close, 75.00
1998-03-06, Close, 76.75
1998-03-09, Close, 83.50
1998-03-10, Close, 83.11
1998-03-11, Close, 79.63
1998-03-12, Close, 78.50
1998-03-13, Close, 78.69
1998-03-16, Close, 79.00
1998-03-17, Close, 79.88
1998-03-18, Close, 85.31
1998-03-19, Close, 84.00
1998-03-20, Close, 84.00
1998-03-23, Close, 83.81
1998-03-24, Close, 84.25
1998-03-25, Close, 81.69
1998-03-26, Close, 82.75
1998-03-27, Close, 85.00
1998-03-30, Close, 85.88
1998-03-31, Close, 85.53
1998-04-0

2001-06-26, Close, 13.25
2001-06-27, Close, 14.04
2001-06-28, Close, 14.36
2001-06-29, Close, 14.15
2001-07-02, Close, 14.53
2001-07-02, BUY CREATE, 14.53
2001-07-03, Close, 14.13
2001-07-03, SELL CREATE, 14.13
2001-07-05, OPERATION PROFIT, GROSS 0.70, NET 0.41
2001-07-05, Close, 15.27
2001-07-05, BUY CREATE, 15.27
2001-07-06, Close, 15.27
2001-07-09, Close, 15.81
2001-07-10, Close, 15.61
2001-07-11, Close, 15.34
2001-07-12, Close, 16.49
2001-07-13, Close, 16.98
2001-07-16, Close, 16.01
2001-07-17, Close, 16.35
2001-07-18, Close, 15.96
2001-07-19, Close, 16.49
2001-07-20, Close, 16.98
2001-07-23, Close, 16.03
2001-07-24, Close, 12.06
2001-07-24, SELL CREATE, 12.06
2001-07-25, OPERATION PROFIT, GROSS -27.10, NET -27.37
2001-07-25, Close, 11.51
2001-07-26, Close, 12.33
2001-07-27, Close, 12.25
2001-07-30, Close, 12.55
2001-07-31, Close, 12.49
2001-08-01, Close, 12.50
2001-08-02, Close, 12.19
2001-08-03, Close, 12.15
2001-08-06, Close, 11.90
2001-08-07, Close, 11.60
2001-08-08, Close, 11.

2004-07-16, Close, 47.10
2004-07-19, Close, 46.02
2004-07-20, Close, 46.99
2004-07-21, Close, 44.76
2004-07-22, Close, 45.82
2004-07-23, Close, 39.98
2004-07-26, Close, 38.78
2004-07-27, Close, 39.11
2004-07-28, Close, 37.97
2004-07-29, Close, 38.58
2004-07-30, Close, 38.92
2004-08-02, Close, 38.30
2004-08-03, Close, 37.61
2004-08-04, Close, 37.12
2004-08-05, Close, 35.61
2004-08-06, Close, 35.49
2004-08-09, Close, 35.32
2004-08-10, Close, 37.10
2004-08-11, Close, 36.56
2004-08-12, Close, 36.29
2004-08-13, Close, 36.13
2004-08-16, Close, 37.95
2004-08-17, Close, 38.23
2004-08-18, Close, 39.36
2004-08-19, Close, 38.63
2004-08-20, Close, 39.51
2004-08-20, BUY CREATE, 39.51
2004-08-23, Close, 39.45
2004-08-24, Close, 39.05
2004-08-25, Close, 40.30
2004-08-26, Close, 40.19
2004-08-27, Close, 39.90
2004-08-30, Close, 38.31
2004-08-31, Close, 38.14
2004-09-01, Close, 38.24
2004-09-02, Close, 39.18
2004-09-03, Close, 38.74
2004-09-07, Close, 38.51
2004-09-08, Close, 38.01
2004-09-08, SELL CRE

2007-05-18, Close, 63.30
2007-05-21, Close, 68.30
2007-05-22, Close, 68.88
2007-05-23, Close, 69.00
2007-05-24, Close, 69.35
2007-05-25, Close, 68.55
2007-05-29, Close, 69.63
2007-05-30, Close, 69.86
2007-05-31, Close, 69.14
2007-06-01, Close, 68.58
2007-06-04, Close, 70.42
2007-06-05, Close, 73.65
2007-06-06, Close, 72.29
2007-06-07, Close, 72.04
2007-06-08, Close, 73.24
2007-06-11, Close, 71.17
2007-06-12, Close, 70.07
2007-06-13, Close, 70.89
2007-06-14, Close, 71.94
2007-06-15, Close, 72.40
2007-06-18, Close, 71.83
2007-06-19, Close, 69.81
2007-06-20, Close, 69.10
2007-06-20, SELL CREATE, 69.10
2007-06-21, OPERATION PROFIT, GROSS 300.70, NET 299.62
2007-06-21, Close, 69.67
2007-06-22, Close, 68.86
2007-06-25, Close, 68.66
2007-06-26, Close, 67.48
2007-06-27, Close, 68.14
2007-06-28, Close, 68.89
2007-06-29, Close, 68.41
2007-07-02, Close, 69.61
2007-07-03, Close, 69.45
2007-07-05, Close, 68.73
2007-07-06, Close, 68.97
2007-07-09, Close, 72.07
2007-07-09, BUY CREATE, 72.07
2007-07-1

2010-03-17, Close, 131.34
2010-03-18, Close, 132.76
2010-03-19, Close, 130.35
2010-03-22, Close, 130.47
2010-03-23, Close, 129.26
2010-03-24, Close, 128.04
2010-03-25, Close, 134.73
2010-03-26, Close, 135.06
2010-03-29, Close, 135.12
2010-03-30, Close, 136.58
2010-03-31, Close, 135.77
2010-04-01, Close, 131.81
2010-04-05, Close, 131.49
2010-04-06, Close, 135.56
2010-04-07, Close, 134.87
2010-04-08, Close, 140.96
2010-04-09, Close, 140.06
2010-04-12, Close, 141.20
2010-04-13, Close, 140.16
2010-04-14, Close, 144.28
2010-04-15, Close, 145.82
2010-04-16, Close, 142.17
2010-04-19, Close, 142.43
2010-04-20, Close, 144.20
2010-04-21, Close, 146.43
2010-04-22, Close, 150.09
2010-04-23, Close, 143.63
2010-04-26, Close, 147.11
2010-04-27, Close, 142.02
2010-04-28, Close, 139.35
2010-04-28, SELL CREATE, 139.35
2010-04-29, OPERATION PROFIT, GROSS 150.80, NET 148.15
2010-04-29, Close, 141.73
2010-04-29, BUY CREATE, 141.73
2010-04-30, Close, 137.10
2010-04-30, SELL CREATE, 137.10
2010-05-03, OPERAT

2013-09-09, Close, 299.71
2013-09-10, Close, 300.36
2013-09-11, Close, 299.64
2013-09-12, Close, 298.86
2013-09-13, Close, 297.92
2013-09-16, Close, 296.06
2013-09-17, Close, 304.17
2013-09-18, Close, 312.03
2013-09-19, Close, 312.06
2013-09-20, Close, 316.34
2013-09-23, Close, 311.49
2013-09-24, Close, 314.13
2013-09-25, Close, 312.65
2013-09-26, Close, 318.12
2013-09-27, Close, 316.01
2013-09-30, Close, 312.64
2013-10-01, Close, 320.95
2013-10-02, Close, 320.51
2013-10-03, Close, 314.76
2013-10-04, Close, 319.04
2013-10-07, Close, 310.03
2013-10-08, Close, 303.23
2013-10-08, SELL CREATE, 303.23
2013-10-09, OPERATION PROFIT, GROSS 97.90, NET 91.93
2013-10-09, Close, 298.23
2013-10-10, Close, 305.17
2013-10-11, Close, 310.89
2013-10-11, BUY CREATE, 310.89
2013-10-14, Close, 310.70
2013-10-15, Close, 306.40
2013-10-15, SELL CREATE, 306.40
2013-10-16, OPERATION PROFIT, GROSS -8.40, NET -14.58
2013-10-16, Close, 310.49
2013-10-16, BUY CREATE, 310.49
2013-10-17, Close, 310.77
2013-10-18, C

2016-12-27, OPERATION PROFIT, GROSS -52.50, NET -67.82
2016-12-27, Close, 771.40
2016-12-27, BUY CREATE, 771.40
2016-12-28, Close, 772.13
2016-12-29, Close, 765.15
2016-12-30, Close, 749.87
2016-12-30, SELL CREATE, 749.87
2017-01-03, OPERATION PROFIT, GROSS -183.30, NET -198.64
2017-01-03, Close, 753.67
2017-01-04, Close, 757.18
2017-01-05, Close, 780.45
2017-01-05, BUY CREATE, 780.45
2017-01-06, Close, 795.99
2017-01-09, Close, 796.92
2017-01-10, Close, 795.90
2017-01-11, Close, 799.02
2017-01-12, Close, 813.64
2017-01-13, Close, 817.14
2017-01-17, Close, 809.72
2017-01-18, Close, 807.48
2017-01-19, Close, 809.04
2017-01-20, Close, 808.33
2017-01-23, Close, 817.88
2017-01-24, Close, 822.44
2017-01-25, Close, 836.52
2017-01-26, Close, 839.15
2017-01-27, Close, 835.77
2017-01-30, Close, 830.38
2017-01-31, Close, 823.48
2017-02-01, Close, 832.35
2017-02-02, Close, 839.95
2017-02-03, Close, 810.20
2017-02-06, Close, 807.64
2017-02-07, Close, 812.50
2017-02-08, Close, 819.71
2017-02-09, Cl

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
df2 = pd.read_csv("AAPL.csv",parse_dates=['Date'], date_parser=dateparse, index_col=['Date'])
print(df2)